<a href="https://colab.research.google.com/github/UdayBattula/pythonnew/blob/main/DEVFOLIO_DeepTrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
from datetime import datetime

# Connect to SQLite (it will create a database file in Colab's file system)
conn = sqlite3.connect('video_credentials.db')
c = conn.cursor()

# Create Users table to store content creator credentials
c.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        joined_at TEXT NOT NULL
    )
''')

# Create Videos table to store video metadata
c.execute('''
    CREATE TABLE IF NOT EXISTS videos (
        video_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        video_title TEXT NOT NULL,
        upload_time TEXT NOT NULL,
        FOREIGN KEY(user_id) REFERENCES users(id)
    )
''')

conn.commit()

# Function to add a content creator
def add_user(name, email):
    try:
        c.execute("INSERT INTO users (name, email, joined_at) VALUES (?, ?, ?)",
                  (name, email, datetime.now()))
        conn.commit()
        print(f"User {name} added successfully.")
    except sqlite3.IntegrityError:
        print(f"Error: A user with the email {email} already exists.")

# Function to upload video metadata
def upload_video(user_email, video_title):
    # Find the user by email
    c.execute("SELECT id FROM users WHERE email = ?", (user_email,))
    user = c.fetchone()

    if user:
        user_id = user[0]
        c.execute("INSERT INTO videos (user_id, video_title, upload_time) VALUES (?, ?, ?)",
                  (user_id, video_title, datetime.now()))
        conn.commit()
        print(f"Video '{video_title}' uploaded successfully.")
    else:
        print("Error: User not found.")

# Example usage:
add_user("John Doe", "john@example.com")
upload_video("john@example.com", "My First Video")

# Fetch all videos to verify
c.execute("SELECT * FROM videos")
print(c.fetchall())

conn.close()


User John Doe added successfully.
Video 'My First Video' uploaded successfully.
[(1, 1, 'My First Video', '2024-10-02 05:35:29.832330')]


In [2]:
!pip install opencv-python-headless ffmpeg-python

In [3]:
import cv2
import ffmpeg

# Function to add watermark to a video
def add_watermark(input_video_path, output_video_path, watermark_text):
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Get video dimensions and properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Output video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Loop through frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Define font, position, scale, and color of watermark
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (10, frame_height - 10)  # Position of watermark
        fontScale = 1
        fontColor = (255, 255, 255)  # White watermark
        thickness = 2
        lineType = cv2.LINE_AA

        # Add watermark text to each frame
        cv2.putText(frame, watermark_text, bottomLeftCornerOfText, font,
                    fontScale, fontColor, thickness, lineType)

        # Write the frame with watermark
        out.write(frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Watermark added. Output video saved at {output_video_path}")

# Example usage
input_video_path = '/content/Tobey-Maguire-Full-Screen-Status-Video.mp4'  # Replace with your actual video file path
output_video_path = '/content/op.mp4'
watermark_text = "toby - My First Video"  # Dynamic watermark for attribution

add_watermark(input_video_path, output_video_path, watermark_text)


Watermark added. Output video saved at /content/op.mp4


In [4]:
import hashlib

# Function to generate a hash of a video file
def generate_video_hash(video_path):
    hash_sha256 = hashlib.sha256()
    with open(video_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

# Example usage
original_video_path = '/content/Tobey-Maguire-Full-Screen-Status-Video.mp4'
tampered_video_path = '/content/op.mp4'

original_hash = generate_video_hash(original_video_path)
tampered_hash = generate_video_hash(tampered_video_path)

print("Original Video Hash:", original_hash)
print("Tampered Video Hash:", tampered_hash)

# Check if the video is tampered
if original_hash == tampered_hash:
    print("No tampering detected.")
else:
    print("Tampering detected!")


Original Video Hash: aa8a31f70ad5e2b5c8f7dbf7e766fa014f47dc1230717d289621d0288725047a
Tampered Video Hash: 3497bd8181ba88faa9af7887e5d55e20cac976b8360f83626a85003eaae48f51
Tampering detected!


In [5]:
!pip install tensorflow tensorflow-addons opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
import cv2
import numpy as np

# Load the pre-trained Xception model
model = Xception(weights='imagenet', include_top=False, pooling='avg')

# Function to preprocess the frame for Xception
def preprocess_frame_xception(frame):
    frame = cv2.resize(frame, (299, 299))  # Xception expects 299x299 input
    frame = image.img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return preprocess_input(frame)  # Preprocess for Xception

# Function to detect tampering in a video using the loaded model
def detect_tampering_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        preprocessed_frame = preprocess_frame_xception(frame)

        # Predict tampering using the model
        features = model.predict(preprocessed_frame)

        # Here you would typically classify the features or use a threshold
        # For this example, we're just displaying the shape of the features
        print("Extracted features shape:", features.shape)

    cap.release()

# Example usage
detect_tampering_in_video('/content/op.mp4')


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Extracted features shape: (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
Extracted fea

In [8]:
# Sample function to verify credentials of the video content creator
def verify_credentials(metadata):
    valid_creators = ["John Doe", "Jane Smith", "Official Channel"]  # Example valid creators
    if metadata['creator'] in valid_creators:
        print(f"Credentials Verified: {metadata['creator']} is a valid creator.")
    else:
        print(f"Credentials Error: {metadata['creator']} is not a recognized creator.")
        return False
    return True

# Sample function to ensure attribution of the video source
def ensure_attribution(metadata):
    valid_sources = ["Official Source", "Trusted Source", "Verified Channel"]  # Example valid sources
    if metadata['source'] in valid_sources:
        print(f"Attribution Verified: The video is from {metadata['source']}.")
    else:
        print(f"Attribution Error: The video source {metadata['source']} is not recognized.")
        return False
    return True

# Modified main function to integrate all phases
def verify_video_integrity(video_path, metadata, model):
    print("Step 1: Verifying Credentials")
    if not verify_credentials(metadata):
        print("Verification failed in Step 1.")
        return

    print("Step 2: Ensuring Attribution")
    if not ensure_attribution(metadata):
        print("Verification failed in Step 2.")
        return

    print("Step 3: Detecting Tampering")
    detect_tampering_in_video(video_path, model)

# Example usage
video_path = '/content/op.mp4'
metadata = {'creator': 'Toby', 'source': 'Official Source'}  # Sample metadata
verify_video_integrity(video_path, metadata, model)


Step 1: Verifying Credentials
Credentials Error: Toby is not a recognized creator.
Verification failed in Step 1.
